In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf

import sys
sys.path.append('/mnt/c/Users/kheut/code/covid19-forecasting/tf_model_1p5/')

from enum import Enum

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense

import tensorflow_probability as tfp
from scipy.stats import beta, truncnorm


# Local imports from model.py, data.py
from model import CovidModel, LogPoissonProb, get_logging_callbacks, Comp, Vax
from data import read_data, create_warmup
#from plots import make_all_plots

import scipy

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20}) # set plot font sizes

2022-01-02 12:19:04.561315: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-02 12:19:04.561373: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
transition_window =3

warmup_start = '20210428'
warmup_end = '20210430'
train_start = '20210501'
train_end = '20210731'
test_start = '20210801'
test_end = '20210831'

state = 'Massachusetts'
state_abbrev = 'MA'

data_dir = '../data'
covid_estim_date = '20210901'
hhs_date = '20210903'
owid_date = '20210903'

log_dir = './logs/new_warmup'


# Learning rate
model_learning_rate = 1e-2
warmup_learning_rate = 20
learning_rate = 1

In [3]:
df = read_data(data_dir=data_dir,
               covid_estim_date=covid_estim_date,
               hhs_date=hhs_date,
               owid_date=owid_date,
               state=state, state_abbrev=state_abbrev)

In [4]:
class Comp(Enum):
    A = 0
    M = 1
    #X = 2
    #G = 3
    
class Vax(Enum):
    total = -1
    no = 0
    yes = 1

In [5]:
# get warmup arrays, splitting on vaccination status
warmup_asymp, warmup_mild, warmup_extreme = create_warmup(df, 
                                                          warmup_start, 
                                                          warmup_end,
                                                          0,0,0)

# re-combine
warmup_asymp = warmup_asymp[Vax.no.value] + warmup_asymp[Vax.yes.value]
warmup_mild = warmup_mild[Vax.no.value] + warmup_asymp[Vax.yes.value]

In [6]:
synth_T_serial = 5.8
synth_rho_M = 0.76
synth_lambda_M = 4.7
synth_nu_M = 3.1

In [7]:
warmup_A_params = {}
warmup_A_params[Vax.total.value] = {}
warmup_A_params[Vax.total.value]['prior'] = []
warmup_A_params[Vax.total.value]['posterior_init'] = []

for day in range(transition_window):
    warmup_A_params[Vax.total.value]['prior'].append({'loc': warmup_asymp[day],
                                                'scale': warmup_asymp[day]/10})
    

In [8]:
x_train = tf.cast(df.loc[train_start:test_end,'Rt'].values, dtype=tf.float32)
y_test = tf.cast(df.loc[train_start:test_end,'mild'], dtype=tf.float32)

2022-01-02 12:19:09.698544: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-02 12:19:09.698627: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-02 12:19:09.698657: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-ODFHK4CF): /proc/driver/nvidia/version does not exist
2022-01-02 12:19:09.699903: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
A_vals = []
M_vals =[]
for day in range(transition_window):
    A_vals.append(warmup_A_params[-1]['prior'][day]['loc'])

for day in range(len(x_train)):
    yesterday_asymp = A_vals[-1]
    
    today_asymp = yesterday_asymp*x_train[day]**(1/synth_T_serial)
    A_vals.append(today_asymp)
    
    today_M = 0
    pi_M=[]
    for j in range(transition_window):
        
        
        lambda_M_fix = synth_lambda_M
        nu_M_fix = synth_nu_M
        poisson_dist = scipy.stats.poisson(lambda_M_fix)
        pi_M_j_ago = poisson_dist.logpmf(j+1)/nu_M_fix
        
        pi_M.append(pi_M_j_ago)
    
    pi_M = scipy.special.softmax(pi_M)
        
    for j in range(transition_window):
        j_ago_asymp = A_vals[day-j-1]
        today_M += j_ago_asymp*synth_rho_M*pi_M[j]
        
    M_vals.append(today_M)

In [10]:

T_serial = {}
T_serial[Vax.total.value] = {}
T_serial[Vax.total.value]['prior'] ={'loc':5.8, 'scale':1}


rho_M = {}
rho_M[Vax.total.value] = {}
rho_M[Vax.total.value]['prior'] = {'a': 31.8, 'b': 10.3}

lambda_M = {}
lambda_M[Vax.total.value] = {}
lambda_M[Vax.total.value]['prior'] = {'loc': 4.7, 'scale': 1}

nu_M = {}
nu_M[Vax.total.value] = {}
nu_M[Vax.total.value]['prior'] = {'loc': 3.1, 'scale': 1.2}




In [11]:
T_serial_scale = 1.0
rho_M_scale = 0.1
lambda_M_scale = 1.0
nu_M_scale = 1.2

warmup_scales = [0.1]*warmup_asymp

In [12]:
T_serial[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(4.0),
                                     'scale':tf.cast(tfp.math.softplus_inverse(T_serial_scale),dtype=tf.float32)}

rho_M[Vax.total.value]['posterior_init'] = {'loc': tf.cast(np.log(0.5/(1-0.5)),dtype=tf.float32),
                                      'scale':tf.cast(tfp.math.softplus_inverse(rho_M_scale),dtype=tf.float32)}

lambda_M[Vax.total.value]['posterior_init'] = {'loc': tf.cast(tfp.math.softplus_inverse(3.0),dtype=tf.float32),
                                         'scale':tf.cast(tfp.math.softplus_inverse(lambda_M_scale),dtype=tf.float32)}

nu_M[Vax.total.value]['posterior_init'] = {'loc': tf.cast(tfp.math.softplus_inverse(5.0),dtype=tf.float32),
                                     'scale':tf.cast(tfp.math.softplus_inverse(nu_M_scale),dtype=tf.float32)}

for day in range(transition_window):
    # must be positive so reverse softplus the mean
    warmup_A_params[Vax.total.value]['posterior_init'].append({'loc': tf.cast(tfp.math.softplus_inverse(2000.0),dtype=tf.float32),
                                                         'scale': tf.cast(tfp.math.softplus_inverse(500.0),dtype=tf.float32)})#tf.cast(tfp.math.softplus_inverse(warmup_asymp[day]/10),dtype=tf.float32)})

model = CovidModel([Vax.total], [Comp.A, Comp.M],
                 transition_window,
                T_serial, rho_M, lambda_M, nu_M,
                 warmup_A_params, posterior_samples=1000, debug_disable_theta=False)

pre_training_preds=tf.reduce_mean(model.call(x_train), axis=-1)

In [13]:
model.variables

2022-01-02 12:19:14.676254: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


[<tf.Variable 'T_serial_A_loc_-1:0' shape=() dtype=float32, numpy=3.9815147>,
 <tf.Variable 'T_serial_A_scale_-1:0' shape=() dtype=float32, numpy=0.54132485>,
 <tf.Variable 'rho_M_loc_-1:0' shape=() dtype=float32, numpy=0.0>,
 <tf.Variable 'rho_M_scale_-1:0' shape=() dtype=float32, numpy=-2.2521684>,
 <tf.Variable 'lambda_M_loc_-1:0' shape=() dtype=float32, numpy=2.9489307>,
 <tf.Variable 'lambda_M_scale_-1:0' shape=() dtype=float32, numpy=0.54132485>,
 <tf.Variable 'nu_M_loc_-1:0' shape=() dtype=float32, numpy=4.9932394>,
 <tf.Variable 'nu_M_scale_-1:0' shape=() dtype=float32, numpy=0.8416177>,
 <tf.Variable 'warmup_A_loc_0_-1:0' shape=() dtype=float32, numpy=2000.0>,
 <tf.Variable 'warmup_A_scale_0_-1:0' shape=() dtype=float32, numpy=500.0>,
 <tf.Variable 'warmup_A_loc_1_-1:0' shape=() dtype=float32, numpy=2000.0>,
 <tf.Variable 'warmup_A_scale_1_-1:0' shape=() dtype=float32, numpy=500.0>,
 <tf.Variable 'warmup_A_loc_2_-1:0' shape=() dtype=float32, numpy=2000.0>,
 <tf.Variable 'warmu

In [14]:
model_variables = model.variables[:-6]
assert all(['warmup' not in variable.name for variable in model_variables])
warmup_variables = model.variables[-6:]
assert all(['warmup' in variable.name for variable in warmup_variables])

In [15]:
loss = LogPoisstf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.onProb() 

model_opt = tf.keras.optimizers.Adam(
    learning_rate=model_learning_rate, #momentum=0.98, nesterov=False, name='SGD'
)
warmup_opt = tf.keras.optimizers.Adam(
    learning_rate=warmup_learning_rate, #momentum=0.98, nesterov=False, name='SGD'
)
grads = tf.gradients(loss, model_variables + warmup_variables )
model_grads = grads[:len(model_variables)]
warmup_grads = grads[len(warmup_variables):]
model_train_op = opt1.apply_gradients(zip(model_grads, model_variables))
warmup_train_op = opt2.apply_gradients(zip(warmup_grads, warmup_variables))
train_op = tf.group(model_train_op, warmup_train_op)

SyntaxError: invalid syntax (341888841.py, line 1)

In [16]:
loss = LogPoissonProb() 
optimizer = tf.keras.optimizers.Adam(
    learning_rate=1, #beta_1=0.1, beta_2=0.1
)

In [17]:
logging_callbacks = get_logging_callbacks('/mnt/c/Users/kheut/logs/covid/bayes_learn_all_estim_01')

In [18]:
model.compile(loss=loss, optimizer=optimizer, run_eagerly=True)
model.fit(x=np.asarray([x_train]), y=np.asarray([y_test]),
         epochs=500, batch_size=0,
        callbacks=logging_callbacks)

Epoch 1/500
1/1 [==============================] - 5s 5s/step - loss: 23103.5098
Epoch 2/500
1/1 [==============================] - 5s 5s/step - loss: 7699.9204
Epoch 3/500
1/1 [==============================] - 5s 5s/step - loss: 6907.1191
Epoch 4/500
1/1 [==============================] - 5s 5s/step - loss: 8425.4473
Epoch 5/500
1/1 [==============================] - 5s 5s/step - loss: 9866.2285
Epoch 6/500
1/1 [==============================] - 5s 5s/step - loss: 10411.9619
Epoch 7/500
1/1 [==============================] - 5s 5s/step - loss: 10673.6729
Epoch 8/500
1/1 [==============================] - 5s 5s/step - loss: 10563.7441
Epoch 9/500
1/1 [==============================] - 5s 5s/step - loss: 9265.0049
Epoch 10/500
1/1 [==============================] - 5s 5s/step - loss: 9117.6328
Epoch 11/500
1/1 [==============================] - 5s 5s/step - loss: 8869.5000
Epoch 12/500
1/1 [==============================] - 5s 5s/step - loss: 9064.0586
Epoch 13/500
1/1 [===============

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(8, 6))
preds=tf.reduce_mean(model.call(x_train), axis=-1)
plt.plot(df.loc[train_start:test_end].index.values, y_test, label='CovidEstim Mild')
plt.plot(df.loc[train_start:test_end].index.values, preds, label='Predicted Mild')
month_ticks = matplotlib.dates.MonthLocator(interval=1)
ax = plt.gca()
ax.xaxis.set_major_locator(month_ticks)
plt.legend()
plt.title('Mild Compartment')

In [ ]:
plt.figure(figsize=(8, 6))
preds=tf.reduce_mean(model.call(x_train), axis=-1)
plt.plot(df.loc[train_start:test_end].index.values, y_test, label='CovidEstim Mild')
plt.plot(df.loc[train_start:test_end].index.values, pre_training_preds, label='Predicted Before Training Mild')
month_ticks = matplotlib.dates.MonthLocator(interval=1)
ax = plt.gca()
ax.xaxis.set_major_locator(month_ticks)
plt.legend()
plt.title('Mild Compartment')

In [ ]:
import tfa.optimizers

In [ ]:
0.8*2000

In [ ]:
warmup_asymp

In [ ]:
warmup_mild

In [ ]:
y_test

In [ ]:
df.loc[warmup_start:'20210510','mild']

In [ ]:
warmup_start

In [ ]:
warmup_asymp

In [ ]:
import copy

In [ ]:
copy_model = copy.deepcopy(model)
copy_model.unconstrained_warmup_A_params[-1][2]['loc'] = tf.Variable(2500.0, dtype=tf.float32)
copy_model.unconstrained_warmup_A_params[-1][1]['loc'] = tf.Variable(2500.0, dtype=tf.float32)
copy_model.unconstrained_warmup_A_params[-1][0]['loc'] = tf.Variable(2500.0, dtype=tf.float32)
copy_model.unconstrained_nu_M[-1]['loc'] = tf.Variable(3.0, dtype=tf.float32)

In [ ]:
plt.figure(figsize=(8, 6))
copy_preds=tf.reduce_mean(copy_model.call(x_train), axis=-1)
plt.plot(df.loc[train_start:test_end].index.values, y_test, label='CovidEstim Mild')
plt.plot(df.loc[train_start:test_end].index.values, copy_preds, label='Predicted Mild')
month_ticks = matplotlib.dates.MonthLocator(interval=1)
ax = plt.gca()
ax.xaxis.set_major_locator(month_ticks)
plt.legend()
plt.title('Mild Compartment')

In [ ]:
loss(y_test, copy_model.call(x_train))

In [ ]:
copy_model.unconstrained_nu_M

In [ ]:
copy_model.unconstrained_rho_M[-1]['loc']

In [ ]:
tf.math.sigmoid(1.2)

In [ ]:
warmup_A_params[-1]

In [ ]:
warmup_asymp

In [ ]:
warmup_mild

In [ ]:
1800/2500

In [ ]:
plt.figure(figsize=(8, 6))
copy_preds=tf.reduce_mean(copy_model.call(x_train), axis=-1)
plt.plot(df.loc[train_start:test_end].index.values, df.loc[train_start:test_end, 'mild']/df.loc[train_start:test_end, 'asymp'], label='Asymp/Mild')
#plt.plot(df.loc[train_start:test_end].index.values, preds, label='Predicted Mild')
month_ticks = matplotlib.dates.MonthLocator(interval=1)
ax = plt.gca()
ax.xaxis.set_major_locator(month_ticks)
plt.legend()
plt.title('Empirical Rho_M')

In [ ]:
0.96**(1/5.2)

In [ ]:
warmup_asymp

In [ ]:
x_train

In [ ]:
model.T_serial_samples_constrained

In [ ]:
copy_model.call(x_train,return_all=True)

In [ ]:
warmup_asymp*x_train[2] ** (1/model.T_serial_samples_constrained)

In [ ]:
copy_model.warmup_A_samples[1]

In [ ]:
warmup_A